In [1]:
#import fwiVis.fwiVis as fv
import s3fs
s3 = s3fs.S3FileSystem(anon=False)
from math import cos, asin, sqrt
import re

import numpy as np
import geopandas as gpd
import pandas as pd
from matplotlib import pyplot as plt
import os
import rioxarray as rio
import xarray as xr
import rasterio
import glob
from shapely.errors import ShapelyDeprecationWarning
from shapely.geometry import Point
import warnings
import folium
import datetime
import time
from folium import plugins
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
import contextily as cx
from shapely.geometry import box
import sys
from datetime import datetime, timedelta

sys.path.insert(0, '/projects/old_shared/fire_weather_vis/base-fwi-vis/')
import fwiVis.fwiVis as fv

In [17]:
### Function fire_timeline

def concat_subsets(files):
    df = []
    for f in files:
        manyfr = pd.read_csv(f)

        manyfr = gpd.GeoDataFrame(manyfr)

        manyfr.t = manyfr.t.astype("datetime64[ns]")
        df.append(manyfr)
    df = pd.concat(df)
    return(df)

def get_lt(lt_string = "Lt_CA_Quebec_"):
    files = glob.glob("/projects/old_shared/fire_weather_vis/Lightning_analysis/computed_data/" + lt_string +"*.csv")
    return(concat_subsets(files))


def fire_timeline(fireID, 
                  lt,
                  year = '2023',
                  path_region="QuebecGlobalNRT_3571" , 
                  check_last = False, 
                  FWI_source = "station" ):
    
    '''
    '''
    
    ## Read in the largefire file of the fireID
    fr = fv.load_large_fire(fireID, year = year, path_region= path_region) ## Cluster of 2 fires. 

        ## TO DO Filter? 
            ## VIIRS Static source filter?
            ## WUI filter? 
            
    ## Subset lightning by time and space
    fr = fr.to_crs("3571")
    lt = gpd.GeoDataFrame(lt, geometry=gpd.points_from_xy(lt.lon, lt.lat), crs=4326) #4674
    lt = lt.to_crs("3571")
    
    ## TO DO: Figure out which CA ecoregion/province the fire is in and subset lighting by that? 
    print("Not yet subseting spatially beyond quebec. Assuming quebec bounding box")
    
    min_threshold = fr.t.astype('datetime64[ns]').min() - timedelta(days = 10)
    possible_lt = lt[lt.t <= fr.t.min()]
    possible_lt = possible_lt[possible_lt.t >= min_threshold]

    oldest_perim = fr[fr.t == fr.t.max()]
    first_perim = fr[fr.t == fr.t.min()]
    first_perim.geometry = first_perim.buffer(750*2) ## Two viirs pixels???
    join_lt = gpd.sjoin(possible_lt, first_perim, predicate = 'within', how = "inner")
    
    #if len(join_lt) == 0:
        
        
    ## Extract "denominator" or the # of trikes from same period
    denominator = possible_lt[possible_lt.t >= join_lt.t_left.min()]
    denominator = denominator[denominator.t <= join_lt.t_left.max()]
    join_lt["num_candidates"] = len(join_lt)
    join_lt["num_strikes"] = len(denominator)
    join_lt["num_strikes_10_days"] = len(possible_lt)
    
    ## Get distance to individuals ignitions
    # fr["perim_rank"] = fr.t.rank()
    # first_geom = fr[fr.perim_rank == 1].geometry
    # first_geom = first_geom.iloc[0]
    # num_starts = len(first_geom.geoms)
    # for i in range(0, num_starts):
    #     join_lt["dist_start_" + str(i)] = join_lt.distance(first_geom.geoms[i].centroid)
    #     print(fr[fr.perim_rank == 1].to_crs("4326").geometry.iloc[0].geoms[i].centroid)
        
    ## Rank candidate by distance
    # range_geoms = list(range(0, num_starts))
    # string = "dist_start_"
    # columns_dists = [string + str(x) for x in range_geoms]
    # top = len(join_lt) * 1 # Top 100%. Could cut to smaller range
    # dist_bool = join_lt[columns_dists].rank() <= top ## NEED a max distance cutoff. 
    # join_lt["candidate"] = dist_bool.any(axis = 1)
    
    ## Get raw VIIRS pixel timing
    date_string = fr.t.astype("datetime64[ns]").max().strftime("%Y%m%d%p")
    raw_obs_times = fv.raw_pixel_times(int(fireID), date_string = date_string)
    raw_obs_times = raw_obs_times.reset_index()
    
    ## get station data
    if(FWI_source == "station"):
        print("Assuming Single Quebec Station. 718270-99999.")
        st = pd.read_csv("s3://veda-data-store-staging/EIS/other/station-FWI/19900101.NRT/FWI/718270-99999.linear.HourlyFWIFromHourlyInterpContinuous.csv") ## Corrected record from Robert
        st.HH = st.HH.astype("int")
        st.YYYY = st.YYYY.astype("int")
        st.MM = st.MM.astype("int")
        st.DD = st.DD.astype("int")
        st = fv.date_convert(st)
        
        st_rm = st[["time", "TEMP_C", 'RH_PERC', 'VPD_HPA', 'WDSPD_KPH',
       'PREC_MM', 'SNOWD_M', 'VIS_KM', 'FFMC', 'DMC', 'DC', 'BUI', 'ISI',
       'FWI', 'OBSMINUTEDIFF_TEMP', 'OBSMINUTEDIFF_RH', 'OBSMINUTEDIFF_WDSPD',
       'ISPRECREPORTED', 'OBSMINUTEDIFF_SNOW', 'OBSMINUTEDIFF_VIS']]
        st_rm = st_rm.rename(columns = {"time":"t"})
        #### Subset station data by time. 
        st_rm = st_rm[st_rm.t >= min_threshold]
        st_rm = st_rm[st_rm.t <= fr.t.max()]
        
    else:
        #print("No other FWI extraction method ready. Sorry. ")
        raise Exception("No other FWI extraction method ready. Sorry. ")
    
    ## Do merging of all dfs 
    foo = join_lt[["InterCloud", "t_left", "lat_left", "lon_left", "current_mag", "error_elps", "num_station"]]
    foo = foo.rename(columns = {"t_left":"t", "lat_left":"lat", "lon_left":"lon"})
    foo.t = foo.t.astype('datetime64[ns]')
    raw_obs_times = raw_obs_times.rename(columns={"count": "viirs_pix_count"}) 
    raw_obs_times.t = raw_obs_times.t.astype("datetime64[ns]")
    merged = foo.merge(raw_obs_times, on = ["t"], how = "outer")
        
    fr_rm = fr.rename(columns = {"lat":"lat_centroid", "lon":"lon_centroid"})
    fr_rm.t = fr_rm.t.astype("datetime64[ns]")
    merged = merged.merge(fr_rm, on = ["t"], how = "outer")
    
    merged = merged.merge(st_rm, on = ["t"], how = "outer")
    merged["fireID"] = fireID
    
    return(merged)
    


    

In [18]:
lt = get_lt()       

tmp = fire_timeline('1462', lt = lt)

tmp
    

['/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/QuebecGlobalNRT_3571/2023/Largefire/F1462_20230825PM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/QuebecGlobalNRT_3571/2023/Largefire/F1462_20230826AM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/QuebecGlobalNRT_3571/2023/Largefire/F1462_20230826PM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/QuebecGlobalNRT_3571/2023/Largefire/F1462_20230827AM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/QuebecGlobalNRT_3571/2023/Largefire/F1462_20230827PM']


/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:584: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lon'] = gdf.centroid.x
/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:585: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf.centroid.y


Not yet subseting spatially beyond quebec. Assuming quebec bounding box


/projects/myenvs/fireatlas_oct4/lib/python3.8/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/projects/myenvs/fireatlas_oct4/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3378: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:920: UserWarning: Unpickling a shapely <2.0 geometry object. Please save the pickle again; shapely 2.1 will not have this compatibility.
  data = pickle.load(file)


Assuming Single Quebec Station. 718270-99999.


,InterCloud,t,lat,lon,current_mag,error_elps,num_station,viirs_pix_count,fireID,n_pixels,...,DC,BUI,ISI,FWI,OBSMINUTEDIFF_TEMP,OBSMINUTEDIFF_RH,OBSMINUTEDIFF_WDSPD,ISPRECREPORTED,OBSMINUTEDIFF_SNOW,OBSMINUTEDIFF_VIS
0,0.0,2023-07-05 20:13:30.534,53.254834,-76.947418,14684.0,23.0,2.0,NaN,1462,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,2023-07-05 21:47:42.789,53.344415,-76.921918,13166.0,16.0,1.0,NaN,1462,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,2023-07-05 21:45:51.032,53.352724,-76.915530,10425.0,20.0,1.0,NaN,1462,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,2023-07-05 13:36:12.107,53.058300,-77.976000,-3775.0,5.0,1.0,NaN,1462,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,2023-07-05 12:02:10.601,52.751981,-78.461306,6228.0,12.0,10.0,NaN,1462,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1712,NaN,2023-08-27 07:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,1462,NaN,...,NaN,NaN,3.041981,7.733160,0.0,0.0,0.0,1.0,0.0,0.0
1713,NaN,2023-08-27 08:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,1462,NaN,...,NaN,NaN,3.041980,7.733159,0.0,0.0,0.0,1.0,0.0,0.0
1714,NaN,2023-08-27 09:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,1462,NaN,...,NaN,NaN,2.863872,7.324979,0.0,0.0,0.0,1.0,0.0,0.0
1715,NaN,2023-08-27 10:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,1462,NaN,...,NaN,NaN,3.284687,8.276314,0.0,0.0,0.0,1.0,0.0,0.0
